In [ ]:
# Works best with jupyter-notebook

In [ ]:
%matplotlib notebook 
#%matplotlib widget 
# https://ipython.readthedocs.io/en/stable/interactive/magics.html
import numpy as np

from spatialmath.base import *

from roboticstoolbox import *
import roboticstoolbox.tools.trajectory as tr

import matplotlib.pyplot as plt
import matplotlib; matplotlib.use("TkAgg") 
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

# Lecture 4.6 - Rotational Interpolation

In our previous section, we learned to interpolate positions. In this section, we turn our attention to interpolating angles. We can't interpolate rotational matrices directly, but we can interpolate vectors of angles like Euler or RPY formats and quaternions as well as we'll see. 

### Roll-Pitch-Yaw Interpolation

Using the toolbox function **jtraj()**, RPY angles can be interpolated. 

jtraj() returns: 
trajectory of coordinates and optionally velocity and acceleration

For example, we can interpolate between (0, 0, 0) & (-π/2, π/2, π/4) radians over 100 time steps:

In [ ]:
q0=[0, 0, 0]
qf=[-np.pi/2, np.pi/2, np.pi/4]

out = tr.jtraj(q0, qf, tv=100)
#print(out.q)

Graph the plot of the interpoloate angles to visualize the smoothness and synchronizity.

In [ ]:
fig1 = plt.figure(1);

plt.xlabel('Time');
plt.ylabel('RPY Angles');
plt.plot(out.q);

### Conversion to Rotation matrices

Sometimes obtaining the rotation matrix equivalent of an RPY angle may be desirable for interpretability or other mathematical computations. We can easily convert beteween RPY and rotation matrices using the *rpy2r* method in the spatialmath toolbox. We can also convert to homogeneous transforms using rpy2tr....

In [ ]:
#The first rotation matrix can be viewed by:
qr1 = out.q[0]
print("The first angle is: ", qr1, "\n")

Convert starting angle:

In [ ]:
R1 = rpy2r(qr1);
print(R1)

Convert final angle:

In [ ]:
#The last rotation matrix can be viewed by:
qr10 = out.q[-1]
print("The last angle is: ", qr10, "\n")

# We can alternatively use each col of qr10
R10 = rpy2r(qr10[0],qr10[1],qr10[2]);
print(R10)


## Tranimate
The RPY sequence can be animated by passing the rotation matrices to the toolbox function tranimate():

### Student exercise: 
1. Work in pairs
2. Solve this tranimate problem (need to work outside jupyter AND you have 15 mins to work on it)
3. Volunteer to share your solution with the class via shared screen.

In [ ]:
import matplotlib; matplotlib.use("TkAgg") #THIS IS THE MAGIC
import matplotlib.pyplot as plt

# TAke the first and last configurations
qr1 = out.q[0]
qrf = out.q[-1]

# Get homogenous transform representations
R1 = rpy2tr(qr1);
Rf = rpy2tr(qrf);

In [ ]:
# Pass final Rf to tranimate
type(Rf)
tranimate(Rf, frame='A', arrow=False, nframes=200, dims=[0,3]);

## Quaternion Interpolation

Interpolation is performed on a great circle on a 4D hypersphere. This is a rotation about a single fixed axis in space which yields the straightest and shortest path between two points. See class resource video for more.

Furthermore, quaternion interpoloation, is represented by a rather complex mathematical equation otherwise known as slerp. Slerp is a function of a scaling parameter s that varies linearly from 0 to 1. We can vary s values to achieve starting and final configurations.

To learn more about slerp in the spatialmath toolbox see:
https://petercorke.github.io/spatialmath-python/func_quat.html?highlight=slerp#spatialmath.base.quaternions.slerp

Note:
For large rotations the path may be the long way around the circle, the option 'shortest' ensures always the shortest path.

In [ ]:
from math import sqrt

q0 = [1/sqrt(2), 1/sqrt(2), 0, 0]  # 90deg rotation about x-axis
q1 = [1/sqrt(2), 0, 1/sqrt(2), 0]  # 90deg rotation about y-axis

We can also use the utility function *qprint()* from the spatialmath.base.quaternions class to easily obtain a quaternion pose using slerp and s.

In [ ]:
#help(qprint)

In [ ]:
qprint(slerp(q0, q1, 0))           # this is q0
qprint(slerp(q0, q1, 1))           # this is q1
qprint(slerp(q0, q1, 0.5))         # this is in "half way" between